In [1]:
"""
Purpose: To visualize how the proofreading performs on the manually proofread cells

Pseucode: 
For the example neuron id
1) get the neuron decomposition
2) Run the error detection to get the errored faces
3) Get the errored synapses
4) Plot them mesh and errored synapses



"""

'\nPurpose: To visualize how the proofreading performs on the manually proofread cells\n\nPseucode: \nFor the example neuron id\n1) get the neuron decomposition\n2) Run the error detection to get the errored faces\n3) Get the errored synapses\n4) Plot them mesh and errored synapses\n\n\n\n'

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path
import neuron_visualizations as nviz


import datajoint_utils as du
from importlib import reload

INFO - 2020-11-30 09:47:43,414 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-30 09:47:43,415 - settings - Setting database.user to celiib
INFO - 2020-11-30 09:47:43,416 - settings - Setting database.password to newceliipass
INFO - 2020-11-30 09:47:43,420 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-30 09:47:43,421 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 09:47:43,431 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-30 09:47:43,628 - settings - Setting enable_python_native_blobs to True


In [3]:
import time
import datajoint_utils as du
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

from datajoint_utils import *

import error_detection as ed

minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-30 09:47:43,647 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 09:47:43,839 - settings - Setting enable_python_native_blobs to True


In [4]:
m65 = minnie
decimation_version = 0
decimation_ratio = 0.25
key_source = (m65.Decimation().proj(decimation_version='version')  &
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)
              & m65.ManualProofreadIDs() & (m65.BaylorSegmentCentroid() & "multiplicity=1").proj())
nucleus_table =(m65.NucleusID & 'nucleus_version=1')  # Aug 1 timestamp
proofread_nucs = nucleus_table & m65.ManualProofreadIDs()
single_soma_nucs = nucleus_table & (dj.U('segment_id').aggr(proofread_nucs, n_nucs='count(nucleus_id)') & 'n_nucs=1').proj()
final_nucs = key_source* single_soma_nucs
final_nucs

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),nucleus_id id of segmented nucleus,nucleus_version segmentation version,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),size (EM voxels) scaled by (4x4x40),svid supervoxel_id of centroid
864691135564655959,0,0.25,225498,1,149712,151648,17549,1315328,85369450370640387
864691135748568361,0,0.25,230236,1,162848,184560,19809,1385955,87203504753932217
864691136194042326,0,0.25,258307,1,174944,137648,21113,1587826,88815663879212610
864691135233108569,0,0.25,262773,1,168096,161952,21503,1314296,87904100020764569
864691135740225387,0,0.25,264870,1,164832,177856,21208,1831649,87413717834430502
864691136105498585,0,0.25,269247,1,177536,213824,19894,1571057,89177746601154881
864691134988386682,0,0.25,269380,1,168016,216288,21534,1688056,87911453071322695
864691136333776819,0,0.25,271518,1,168464,226192,21121,1666190,87912758674384086
864691135771629819,0,0.25,294545,1,182976,135728,21116,1462702,89941288908168255
864691135748575017,0,0.25,296726,1,180976,143888,22162,1827886,89660913577301286


In [5]:
eligible_table = minnie.Decomposition & final_nucs.proj()

In [6]:
seg_ids_to_test = eligible_table.fetch("segment_id")
seg_ids_to_test

array([864691135564655959, 864691135748568361, 864691136194042326,
       864691135233108569, 864691135740225387, 864691136105498585,
       864691134988386682, 864691136333776819, 864691135771629819,
       864691136105484249, 864691135212632448, 864691136296650011,
       864691136311740477, 864691135491233631, 864691135945413668,
       864691135393964789, 864691135113167769, 864691136105493209,
       864691135272164113, 864691136618403213, 864691135974454639,
       864691136003930314, 864691135510455760, 864691136537220258,
       864691135785277636, 864691136333787571, 864691136105491417,
       864691135645592260, 864691136056391384, 864691136099807093,
       864691135348272855, 864691135737374100, 864691135212690816,
       864691136550489250, 864691136194103510, 864691135589907979,
       864691135407289801, 864691136545546146, 864691135382947307,
       864691136008425132, 864691135925564174, 864691136333790899,
       864691135974454895, 864691136008573614, 864691136134446

# Defining the Table to Store the statistics

In [7]:
@schema
class ValidationStats(dj.Manual):
    definition="""
    -> minnie.ManualProofreadIDs
    ---
    new_segment_id: bigint unsigned

    true_positive_presyn: int
    false_positive_presyn: int
    true_negative_presyn: int
    false_negative_presyn: int

    precision_presyn=NULL: double
    recall_presyn=NULL: double
    f1_presyn=NULL: double

    true_positive_postsyn: int
    false_positive_postsyn: int
    true_negative_postsyn: int
    false_negative_postsyn: int

    precision_postsyn=NULL: double
    recall_postsyn=NULL: double
    f1_postsyn=NULL: double

    true_positive_total : int
    false_positive_total : int
    true_negative_total : int
    false_negative_total : int

    precision_total=NULL: double
    recall_total=NULL: double
    f1_total=NULL: double

    """

In [8]:
plot_flag = False

In [9]:
for segment_id in seg_ids_to_test:
    # Doing the Actual Function


    key = dict(segment_id=segment_id)
    print(key)

    segment_id = key["segment_id"]
    global_start = time.time()

    verbose = True

    print(f"\n\n\n\n----- Working on {segment_id}-------")
    whole_pass_time = time.time()

    neuron_obj = (minnie.Decomposition() & key).fetch1("decomposition")
    errored_faces = ed.error_faces_by_axons(neuron_obj,verbose=True,visualize_errors_at_end=False)

    print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")

    # Visualize the Actual Neuron

    if plot_flag:
        print(neuron_obj.segment_id)
        nviz.plot_objects(main_mesh=neuron_obj.mesh,
                     mesh_alpha=1)

    # Visualize the Results

    print(f"n_errored_faces = {len(errored_faces)}")

    ed = reload(ed)


    if neuron_obj is None:
        neuron_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")

    current_mesh = neuron_obj.mesh

    error_submesh = current_mesh.submesh([errored_faces],append=True)
    valid_mesh = tu.subtract_mesh(current_mesh,error_submesh)

    if plot_flag:
        nviz.plot_objects(main_mesh=valid_mesh,
                     meshes=error_submesh,
                      meshes_colors="red",
                     mesh_alpha=1)

    # How to get the percentage of synapses removed

    """
    Pseudocode: 
    1) Get the synapse file that corresponds to it
    2) Get the matching v7 id
    3) Get the v7 synapses (pre and post) that are still there
    4) Get the pre and post that are stil their after autoproofreading
    5) Things to find:
    - Eliminated and should have been (true positives)
    - Eliminated but SHOULDN'T (false positives) 
    - Not Eliminated and should have been not (true negative)
    - Not Eliminated but should have been eliminated (false negatie) 

    """

    # 1) Get the synapse file that corresponds to it
    data_path = Path("./manual_proofread_data/")
    current_file = data_path / Path(f"{segment_id}.pkl")
    current_file

    import system_utils as su
    curr_synapse_table = su.load_object(current_file)
    #eliminating the self connections
    df = curr_synapse_table[curr_synapse_table["pre_pt_root_id_Aug"] != curr_synapse_table["post_pt_root_id_Aug"]]
    df

    #1b) Get the synapse data for the august neuron 
    ctr_pt_position = df["ctr_pt_position"].to_numpy()
    synapse_ids = df["id"].to_numpy()
    synapse_centers = np.vstack(ctr_pt_position)
    synapse_ids.shape,synapse_centers.shape
    timestamps = np.zeros(len(synapse_ids))

    # 2) Get the matching v7 id
    import pandas as pd
    aug_to_v7_table = pd.read_csv("./proofread_cells_seg_correspondence_Aug1-v7.csv")
    v7_id = aug_to_v7_table[aug_to_v7_table["old_seg_id"]==segment_id]["new_seg_id"].tolist()[0]
    v7_id

    # check that the new v7 segment id is in the synapse table

    def get_pd_id_data(col_name,return_unique=False):
        ret_data = df [col_name].to_numpy()
        if return_unique:
            return np.unique(ret_data)
        else:
            return ret_data

    pre_pt_root_id = get_pd_id_data("pre_pt_root_id")
    pre_pt_root_id.shape,v7_id in pre_pt_root_id

    post_pt_root_id = get_pd_id_data("post_pt_root_id")
    post_pt_root_id.shape,v7_id in post_pt_root_id

    original_presyn = df[(df["pre_pt_root_id_Aug"] == segment_id)]["id"].to_numpy()
    original_postsyn = df[(df["post_pt_root_id_Aug"] == segment_id)]["id"].to_numpy()

    original_presyn.shape,original_postsyn.shape

    #3) Get the v7 synapses (pre and post) that are still there
    manual_remaining_presyn = df[(df["pre_pt_root_id"] == v7_id)]["id"].to_numpy()
    manual_remaining_postsyn = df[(df["post_pt_root_id"] == v7_id)]["id"].to_numpy()
    manual_remaining_presyn.shape,manual_remaining_postsyn.shape

    # find the non-remaining presyns and postsyns
    manual_error_presyn = np.setdiff1d(original_presyn,manual_remaining_presyn)
    manual_error_postsyn = np.setdiff1d(original_postsyn,manual_remaining_postsyn)

    manual_error_presyn.shape,manual_error_postsyn.shape

    len(manual_remaining_presyn) + len(manual_error_presyn), len(manual_remaining_postsyn) + len(manual_error_postsyn)

    #4) Get the pre and post that are stil their after autoproofreading
    ed = reload(ed)
    err_synapse_ids,non_err_synapse_ids = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                         errored_faces,return_synapse_ids=True,
                                                                       synapse_centers=synapse_centers,
                                                                       synapse_ids=synapse_ids
                                                                      )

    # divided the valid synapses into presyn and postsyn
    auto_remaining_presyn = np.intersect1d(original_presyn,non_err_synapse_ids)
    auto_remaining_postsyn = np.intersect1d(original_postsyn,non_err_synapse_ids)

    #dividing the error synapses into presyn and postsyn
    auto_error_presyn = np.intersect1d(original_presyn,err_synapse_ids)
    auto_error_postsyn = np.intersect1d(original_postsyn,err_synapse_ids)

    len(auto_remaining_presyn), len(auto_error_presyn), len(auto_remaining_postsyn), len(auto_error_postsyn)

    # Calculating the final scores

    # doing the presyn analysis
    true_positive_presyn = np.intersect1d(manual_error_presyn,auto_error_presyn) #deemed an error and was an error
    false_positive_presyn = np.intersect1d(manual_remaining_presyn,auto_error_presyn)#deemed an error but not an error

    true_negative_presyn = np.intersect1d(manual_remaining_presyn,auto_remaining_presyn) #not deemed an error and should not be an error
    false_negative_presyn = np.intersect1d(manual_error_presyn,auto_remaining_presyn) #not deemed and error but should be an error

    len(true_positive_presyn),len(false_positive_presyn),len(true_negative_presyn),len(false_negative_presyn)

    np.sum([len(true_positive_presyn),len(false_positive_presyn),len(true_negative_presyn),len(false_negative_presyn)])

    # Doing the postsyn analysis
    true_positive_postsyn = np.intersect1d(manual_error_postsyn,auto_error_postsyn) #deemed an error and was an error
    false_positive_postsyn = np.intersect1d(manual_remaining_postsyn,auto_error_postsyn)#deemed an error but not an error

    true_negative_postsyn = np.intersect1d(manual_remaining_postsyn,auto_remaining_postsyn) #not deemed an error and should not be an error
    false_negative_postsyn= np.intersect1d(manual_error_postsyn,auto_remaining_postsyn) #not deemed and error but should be an error

    len(true_positive_postsyn),len(false_positive_postsyn),len(true_negative_postsyn),len(false_negative_postsyn)

    np.sum([len(true_positive_postsyn),len(false_positive_postsyn),len(true_negative_postsyn),len(false_negative_postsyn)])

    #
    true_positive_total = list(true_positive_presyn) + list(true_positive_postsyn) 
    false_positive_total = list(false_positive_presyn) + list(false_positive_postsyn)

    true_negative_total = list(true_negative_presyn) + list(true_negative_postsyn)
    false_negative_total = list(false_negative_presyn) + list(false_negative_postsyn)

    len(true_positive_total),len(false_positive_total),len(true_negative_total),len(false_negative_total)

    np.sum([len(true_positive_total),len(false_positive_total),len(true_negative_total),len(false_negative_total)])

    def precision(TP,FP):
        if (TP + FP) > 0:
            return TP/(TP + FP)
        else:
            return np.nan

    def recall(TP,FN):
        if (TP + FN) > 0:
            return TP/(TP + FN)
        else:
            return np.nan

    def f1(TP,FP,FN):
        curr_prec = precision(TP,FP)
        curr_recall = recall(TP,FN)

        if curr_prec + curr_recall > 0:
            return 2*(curr_prec*curr_recall)/(curr_prec + curr_recall)
        else:
            return np.nan

    def calculate_scores(TP,FP,FN):
        return dict(precision=precision(TP,FP),
                   recall=recall(TP,FN),
                   f1=f1(TP,FP,FN))

    # calculating the accuracy, precision and f1

    #precision = (true positives)/(true positives + false positives)
    presyn_scores =  calculate_scores(len(true_positive_presyn),len(false_positive_presyn),len(false_negative_presyn))
    presyn_scores

    postsyn_scores =  calculate_scores(len(true_positive_postsyn),len(false_positive_postsyn),len(false_negative_postsyn))
    postsyn_scores

    total_scores =  calculate_scores(len(true_positive_total),len(false_positive_total),len(false_negative_total))
    total_scores

    # Visualize the Results


    err_synapses,non_err_synapses = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                         errored_faces,return_synapse_centroids=True,
                                                                       synapse_centers=synapse_centers,
                                                                       synapse_ids=synapse_ids
                                                                      )

    print(f"Segment ID = {segment_id}")
    print(f"n_synapses = {len(err_synapses) + len(non_err_synapses)}, n_errored_synapse = {len(err_synapses)}")


    valid_synapse_color = "yellow"
    error_color = "red"
    scatters_list = []
    scatters_colors = []

    if len(err_synapses) > 0:
        scatters_list.append(err_synapses)
        scatters_colors.append(error_color)
    if len(non_err_synapses) > 0:
        scatters_list.append(non_err_synapses)
        scatters_colors.append(valid_synapse_color)

    if plot_flag:
        if len(scatters_list) > 0:
            nviz.plot_objects(main_mesh=valid_mesh,
                      meshes=[error_submesh],
                              mesh_alpha=1,
                      meshes_colors=[error_color],
                    scatters=[err_synapses,non_err_synapses],
                     scatters_colors=[error_color,valid_synapse_color],
                             scatter_size=0.2)

        else:
            nviz.plot_objects(main_mesh=valid_mesh,
                     meshes=error_submesh,
                      meshes_colors=error_color,
                     mesh_alpha=1)


    # Write the Results to a Datajoint Table


    info_key = dict(segment_id=segment_id,
                   new_segment_id=v7_id,

                    true_positive_presyn = len(true_positive_presyn),
                    false_positive_presyn = len(false_positive_presyn),
                    true_negative_presyn= len(true_negative_presyn),
                    false_negative_presyn = len(false_negative_presyn),

                    precision_presyn = presyn_scores["precision"],
                    recall_presyn = presyn_scores["recall"],
                    f1_presyn = presyn_scores["f1"],

                    true_positive_postsyn = len(true_positive_postsyn),
                    false_positive_postsyn = len(false_positive_postsyn),
                    true_negative_postsyn = len(true_negative_postsyn),
                    false_negative_postsyn = len(false_negative_postsyn),

                    precision_postsyn = postsyn_scores["precision"],
                    recall_postsyn = postsyn_scores["recall"],
                    f1_postsyn = postsyn_scores["f1"],

                    true_positive_total= len(true_positive_total),
                    false_positive_total = len(false_positive_total),
                    true_negative_total = len(true_negative_total),
                    false_negative_total = len(false_negative_total),

                    precision_total = total_scores["precision"],
                    recall_total=total_scores["recall"],
                    f1_total = total_scores["f1"],


                   )
    print(f"total_scores = {total_scores}")

    ValidationStats.insert1(info_key,skip_duplicates=True)

{'segment_id': 864691135564655959}




----- Working on 864691135564655959-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f03f0bb7840>, <function median_mesh_center at 0x7f03f0bb7ae8>, <function n_spines at 0x7f03f0bb7950>, <function n_faces_branch at 0x7f03f2d20510>, <function skeleton_distance_branch at 0x7f03f0bb78c8>, <function spines_per_skeletal_length at 0x7f03f0bb7d90>, <function no_spine_median_mesh_center at 0x7f03f0bb7bf8>]
functions_list = [<function axon_segment at 0x7f03f0bb5488>]
Axons not keeping because of soma: {'L0': array([17, 18, 20, 21])}

----- Working on L1 ------


For limb 1 the valid axon branches are []
The following are not valid: []

----- Working on L4 ------
-- Axon Group 0 of size 35--
   Working on soma 0, starting_node 34
Not using AIS angle threshold
sk_angle= 16.380534881177304


For limb 4 the valid axon branches are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 2

Not using AIS angle threshold
sk_angle= 13.39553702962113


For limb 3 the valid axon branches are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]
The following are not valid: []
final_error_axons = {'L0': array([], dtype=int64), 'L1': [], 'L2': [], 'L3': []}


 -------- Total number of error faces = 0 --------------


 ------ Total time for 864691135233108569 = 16.69904851913452 ------
n_errored_faces = 0
Segment ID = 864691135233108569
n_synapses = 792, n_errored_synapse = 0
total_scores = {'precision': nan, 'recall': 0.0, 'f1': nan}
{'segment_id': 864691135740225387}




----- Working on 864691135740225387-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f03f0bb7840>, <function median_mesh_center at 0x7f03f0bb7ae8>, <function n_spines at 0x7f03f0bb7950>, <function n_faces_branch at 0x7f03f2d20510>, <function skeleton_distance_branch at 0x7f03f0bb78c8>, <function spines_per_skeleta

Axons not keeping because of soma: {'L0': array([16, 17, 18, 19, 20, 22, 23])}

----- Working on L1 ------


For limb 1 the valid axon branches are []
The following are not valid: []

----- Working on L2 ------
-- Axon Group 0 of size 5--
   Working on soma 0, starting_node 10
Not using AIS angle threshold
sk_angle= 44.13514683886762
-- Axon Group 1 of size 37--
   Working on soma 0, starting_node 10
Not using AIS angle threshold
sk_angle= 44.13514683886762
-- Axon Group 2 of size 15--
   Working on soma 0, starting_node 10
Not using AIS angle threshold
sk_angle= 44.13514683886762


For limb 2 the valid axon branches are [67 68 69 70 71 12 13 14 15 20 21 22 24 25 26 27 28 34 35 36 37 38 40 41
 42 44 45 46 48 49 54 55 56 57 58 59 60 62 63 64 65 66 32 39 43 47 17 18
 19 52 51 53 23 50 29 30 31]
The following are not valid: []

----- Working on L3 ------
-- Axon Group 0 of size 54--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 33.51912864571277
-- Axon Gro



 -------- Total number of error faces = 82075 --------------


 ------ Total time for 864691136105484249 = 28.83038830757141 ------
n_errored_faces = 82075
Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691136105484249
n_synapses = 1990, n_errored_synapse = 251
total_scores = {'precision': 0.3705179282868526, 'recall': 0.37349397590361444, 'f1': 0.372}
{'segment_id': 864691135212632448}




----- Working on 864691135212632448-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f03f0bb7840>, <function median_mesh_center at 0x7f03f0bb7ae8>, <function n_spines at 0x7f03f0bb7950>, <function n_faces_branch at 0x7f03f2d20510>, <function skeleton_distance_branch at 0x7f03f0bb78c8>, <function spines_per_skeletal_length at 0x7f03f0bb7d90>, <function no_spine_median_mesh_center at 0x7f03f0bb7bf8>]
functions_list = 



 -------- Total number of error faces = 92807 --------------


 ------ Total time for 864691136296650011 = 55.34755825996399 ------
n_errored_faces = 92807
Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691136296650011
n_synapses = 3311, n_errored_synapse = 162
total_scores = {'precision': 0.9444444444444444, 'recall': 0.26936619718309857, 'f1': 0.4191780821917808}
{'segment_id': 864691136311740477}




----- Working on 864691136311740477-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f03f0bb7840>, <function median_mesh_center at 0x7f03f0bb7ae8>, <function n_spines at 0x7f03f0bb7950>, <function n_faces_branch at 0x7f03f2d20510>, <function skeleton_distance_branch at 0x7f03f0bb78c8>, <function spines_per_skeletal_length at 0x7f03f0bb7d90>, <function no_spine_median_mesh_center at 0x7f03f0bb7bf8>]
func

...



 -------- Total number of error faces = 77372 --------------


 ------ Total time for 864691135736387732 = 64.61205315589905 ------
n_errored_faces = 77372
Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691135736387732
n_synapses = 4379, n_errored_synapse = 115
total_scores = {'precision': 0.6869565217391305, 'recall': 0.47305389221556887, 'f1': 0.5602836879432624}
{'segment_id': 864691135501578306}




----- Working on 864691135501578306-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f03f0bb7840>, <function median_mesh_center at 0x7f03f0bb7ae8>, <function n_spines at 0x7f03f0bb7950>, <function n_faces_branch at 0x7f03f2d20510>, <function skeleton_distance_branch at 0x7f03f0bb78c8>, <function spines_per_skeletal_length at 0x7f03f0bb7d90>, <function no_spine_median_mesh_center at 0x7f03f0bb7bf8>]
func

Not using AIS angle threshold
sk_angle= 3.6144283652038647
-- Axon Group 3 of size 3--
   Working on soma 0, starting_node 27
Not using AIS angle threshold
sk_angle= 3.6144283652038647


For limb 7 the valid axon branches are [28 29 31  3  4  5  6  7  8  9 10 11 12 13 21 23 25 18 14 15 24 19 20]
The following are not valid: []

----- Working on L8 ------


For limb 8 the valid axon branches are []
The following are not valid: []
final_error_axons = {'L0': array([38, 40, 55, 57, 69]), 'L2': [], 'L3': [9], 'L6': [], 'L7': [], 'L8': []}


 -------- Total number of error faces = 43539 --------------


 ------ Total time for 864691136201041854 = 49.62494659423828 ------
n_errored_faces = 43539
Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691136201041854
n_synapses = 4706, n_errored_synapse = 158
total_scores = {'precision': 0.056962025316455694, 'recall': 0.2, '

Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691135735426708
n_synapses = 3713, n_errored_synapse = 94
total_scores = {'precision': 0.8085106382978723, 'recall': 0.07991587802313355, 'f1': 0.14545454545454545}
{'segment_id': 864691136618412685}




----- Working on 864691136618412685-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f03f0bb7840>, <function median_mesh_center at 0x7f03f0bb7ae8>, <function n_spines at 0x7f03f0bb7950>, <function n_faces_branch at 0x7f03f2d20510>, <function skeleton_distance_branch at 0x7f03f0bb78c8>, <function spines_per_skeletal_length at 0x7f03f0bb7d90>, <function no_spine_median_mesh_center at 0x7f03f0bb7bf8>]
functions_list = [<function axon_segment at 0x7f03f0bb5488>]
Axons not keeping because of soma: {'L0': array([8])}

----- Working on L1 ------


For limb 1 the va

Not using AIS angle threshold
sk_angle= 71.55371166059936
*****Path to axon group not valid******


For limb 2 the valid axon branches are []
The following are not valid: [8, 9, 10, 11, 14, 15, 17]

----- Working on L3 ------


For limb 3 the valid axon branches are []
The following are not valid: []

----- Working on L4 ------
-- Axon Group 0 of size 3--
   Working on soma 0, starting_node 0
Not using AIS angle threshold
sk_angle= 99.74865833239323
*****Path to axon group not valid******
-- Axon Group 1 of size 1--
   Working on soma 0, starting_node 0
Not using AIS angle threshold
sk_angle= 99.74865833239323
*****Path to axon group not valid******


For limb 4 the valid axon branches are []
The following are not valid: [15, 17, 18, 20]

----- Working on L6 ------


For limb 6 the valid axon branches are []
The following are not valid: []

----- Working on L7 ------


For limb 7 the valid axon branches are []
The following are not valid: []
final_error_axons = {'L0': array([ 45,  47, 



 -------- Total number of error faces = 11189 --------------


 ------ Total time for 864691136056340440 = 89.72723269462585 ------
n_errored_faces = 11189
Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691136056340440
n_synapses = 5362, n_errored_synapse = 48
total_scores = {'precision': 0.0, 'recall': nan, 'f1': nan}


In [11]:
ValidationStats()

segment_id segment id for those to be decimated,new_segment_id,true_positive_presyn,false_positive_presyn,true_negative_presyn,false_negative_presyn,precision_presyn,recall_presyn,f1_presyn,true_positive_postsyn,false_positive_postsyn,true_negative_postsyn,false_negative_postsyn,precision_postsyn,recall_postsyn,f1_postsyn,true_positive_total,false_positive_total,true_negative_total,false_negative_total,precision_total,recall_total,f1_total
864691134988386682,864691135278269857,35,0,19,244,1.0,0.12544802867383512,0.22292993630573246,0,0,1862,14,nan,0.0,nan,35,0,1881,258,1.0,0.11945392491467577,0.21341463414634146
864691135113167769,864691135367058169,78,2,22,105,0.975,0.4262295081967213,0.5931558935361216,5,156,2085,9,0.031055900621118012,0.35714285714285715,0.05714285714285714,83,158,2107,114,0.34439834024896265,0.4213197969543147,0.3789954337899544
864691135212632448,864691135446675028,254,2,27,130,0.9921875,0.6614583333333334,0.7937500000000001,7,9,3336,7,0.4375,0.5,0.4666666666666667,261,11,3363,137,0.9595588235294118,0.6557788944723618,0.7791044776119402
864691135212690816,864691135775906989,0,0,16,31,nan,0.0,nan,0,0,2334,2,nan,0.0,nan,0,0,2350,33,nan,0.0,nan
864691135233108569,864691136023889209,0,0,12,47,nan,0.0,nan,0,0,731,2,nan,0.0,nan,0,0,743,49,nan,0.0,nan
864691135272164113,864691136194298600,61,0,20,133,1.0,0.31443298969072164,0.4784313725490196,0,11,5636,18,0.0,0.0,nan,61,11,5656,151,0.8472222222222222,0.28773584905660377,0.4295774647887324
864691135348272855,864691136143561524,342,0,12,97,1.0,0.7790432801822323,0.8758002560819462,4,31,5967,21,0.11428571428571428,0.16,0.13333333333333333,346,31,5979,118,0.9177718832891246,0.7456896551724138,0.8228299643281808
864691135382947307,864691135593659947,13,0,50,20,1.0,0.3939393939393939,0.5652173913043478,1,34,2599,0,0.02857142857142857,1.0,0.05555555555555556,14,34,2649,20,0.2916666666666667,0.4117647058823529,0.34146341463414637
864691135393964789,864691137019596142,133,0,14,68,1.0,0.6616915422885572,0.7964071856287425,11,0,1592,96,1.0,0.102803738317757,0.1864406779661017,144,0,1606,164,1.0,0.4675324675324675,0.6371681415929203
864691135407289801,864691135848030814,184,0,9,84,1.0,0.6865671641791045,0.8141592920353982,4,203,3124,3,0.01932367149758454,0.5714285714285714,0.037383177570093455,188,203,3133,87,0.48081841432225064,0.6836363636363636,0.5645645645645645
